In [2]:
import pandas as pd
import numpy as np
from scipy import stats

# 1. Carregamento dos Dados
# Certifique-se de que os caminhos '../data/raw/...' estão corretos para o seu sistema
df_orders = pd.read_csv('../data/raw/FACT_Orders.csv')
df_del = pd.read_csv('../data/raw/DIM_Delivery.csv')

# 2. Tratamento e Merge (O passo que faltava)
# Ordenar para garantir integridade
df_orders = df_orders.sort_values('Id').reset_index(drop=True)
df_del = df_del.sort_values('Id').reset_index(drop=True)

# Conversão de Datas (Crucial para a comparação funcionar)
df_orders['Order_Date'] = pd.to_datetime(df_orders['Order_Date'], errors='coerce')
df_del['D_Date'] = pd.to_datetime(df_del['D_Date'], errors='coerce')

# UNIR AS TABELAS (Merge)
# Juntamos Pedidos (df_orders) com Entregas (df_del) usando o 'Id' como chave
df = pd.merge(df_orders, df_del, on='Id', how='left')

# Agora 'df' tem as colunas 'D_Date' (do Delivery) e 'Order_Date' (do Orders)

# 3. Saneamento (A Regra de Ouro)
mask_limpeza = (df['D_Date'] >= df['Order_Date']) & \
               ~((df['Purchase_Status'] == 'Cancelado') & (df['Status'] == 'Entregue'))

df_clean = df[mask_limpeza].copy()

# 4. Cálculo de IC 95% (Exemplo)
def confidence_interval(data):
    # Remove NaNs antes de calcular para evitar erros
    data = data.dropna()
    if len(data) < 2: return (np.nan, np.nan)

    m, se = np.mean(data), stats.sem(data)
    h = se * stats.t.ppf((1 + 0.95) / 2., len(data)-1)
    return m-h, m+h

# Calculando IC do Ticket Médio (apenas vendas não canceladas)
vendas_validas = df_clean[df_clean['Purchase_Status'] != 'Cancelado']['Total']
ci_ticket = confidence_interval(vendas_validas)

print(f"Base Saneada: {len(df_clean)} registros")
print(f"IC Ticket Médio (95%): R$ {ci_ticket[0]:.2f} a R$ {ci_ticket[1]:.2f}")

Base Saneada: 1550 registros
IC Ticket Médio (95%): R$ 2401.33 a R$ 2765.40
